In [6]:
import pandas as pd
import datetime
import time




In [21]:

movies_MD_df = pd.read_csv("data\movies_metadata.csv")
credits_df = pd.read_csv("data\credits.csv")

C:\Users\oferinbar\AppData\Local\Temp\ipykernel_23056\4229644854.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_MD_df = pd.read_csv("data\movies_metadata.csv")


In [22]:
# drop unnecessary columns
movies_MD_df = movies_MD_df.drop(
    columns=[
        "adult",
        "belongs_to_collection",
        "homepage",
        "poster_path",
        "production_companies",
        "production_countries",
        "spoken_languages",
        "status",
        "tagline",
        "video",
        "budget",
        "revenue",
        "vote_average",
        "popularity",
    ]
)
# Remove rows which have NaN values
movies_MD_df = df.dropna()
credits_df = credits_df.dropna()
# print row where id is 1997-08-20
# change the id data type to int
movies_MD_df["id"] = movies_MD_df["id"].astype(int)
# merge on the id column
movies_creds_df = movies_MD_df.merge(credits_df, on="id")

# print how many items we have
print("Number of items: ", len(movies_creds_df))



Number of items:  44646


In [28]:
df= movies_creds_df.copy(deep=True)
#  convert to Unix TS
df["release_date"] = (pd.to_datetime(df["release_date"], yearfirst= True )  - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

,genres,id,imdb_id,original_language,original_title,overview,release_date,runtime,title,vote_count,cast_x,crew_x,cast_y,crew_y
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",815011200,81.0,Toy Story,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,818985600,104.0,Jumanji,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,819590400,101.0,Grumpier Old Men,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",819590400,127.0,Waiting to Exhale,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,"[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,792374400,106.0,Father of the Bride Part II,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
